# KB 추천시스템 Lecture

## Libarary Setting

In [0]:
import numpy as np
import os
import sys
import tensorflow as tf
import pandas as pd
import requests

In [0]:
# # DataDownload 
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_car_master_read.tsv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_car_master.tsv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_car_reservation.csv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_car_reservation.csv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_car_wish.csv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_member.csv'
# !wget 'https://pai-lecture.s3.ap-northeast-2.amazonaws.com/KB-ChaChaCha_Recsys/KB_%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AD%E1%84%8B%E1%85%B2%E1%86%A8/data/tb_sms.csv'

## Data Load

In [0]:
def dataload(root_dir):
    """
    Description:
        필요한 데이터를 불러옵니다.
    Args:
        root_dir: data 가 존재하는 폴더
    Returns:
        code_df, DataFrame,  코드 정보가 들어가 있는 dataframe
        cvr_dfs, dict, 이름, data 가 아래 구조로 되어 있는 형태의 dict로 반환.
        {
            'tb_sms': {'data':   DataFrame},
            'tb_call_request': {'data':   DataFrame},
            'tb_car_wish': {'data':   DataFrame},
            'tb_car_reservation': {'data':   DataFrame}
        }
        ctr_dfs, dict, 이름, data 가 아래 구조로 되어 있는 형태의 dict로 반환.
        {
            'tb_car_master_read': {'data':   DataFrame},
        }
    """
    # tb_car_master TABLE : 차량 매물 정보에 대한 tsv 파일
    # 해당 tsv 에는 CAR_CODE 가 포함 되어 있습니다
    code_df = pd.read_csv(os.path.join(root_dir, 'tb_car_master.tsv'), sep='\t', encoding='utf-16', low_memory=False)
    code_column = 'CAR_CODE'
    chosen_columns = ['CAR_SEQ', code_column]
    code_df = code_df.loc[:, chosen_columns]

    # tb_car_master_read TABLE : 차량 조회 로그에 대한 tsv 파일
    # 저장된 .tsv 파일들을 불러옵니다
    # request

    request_df = pd.read_csv(os.path.join(root_dir, 'tb_call_request.tsv'), sep='\t', encoding='UTF-16')
    reservation_df = pd.read_csv(os.path.join(root_dir, 'tb_car_reservation.csv'), encoding='UTF-8')
    wish_df = pd.read_csv(os.path.join(root_dir, 'tb_car_wish.csv'), encoding='UTF-8', low_memory=False)
    car_master_read_df = pd.read_csv(os.path.join(root_dir, 'tb_car_master_read.tsv'), sep='\t',
                                     encoding='utf-16', low_memory=False)
    # sms preprocessing 합니다.
    sms_df = pd.read_csv(os.path.join(root_dir, 'tb_sms.csv'), encoding='UTF-8')
    sms_df.columns = ['SMS_SEQ', 'SMS_KIND', 'CAR_SEQ', 'MEMBER_NO', 'REGI_DATE']

    cvr_dfs = {'tb_call_request': {'data': request_df},
               'tb_car_reservation': {'data': reservation_df},
               'tb_car_wish': {'data': wish_df},
               'tb_sms': {'data': sms_df}}
    ctr_dfs = {'car_master_read_df': {'data': car_master_read_df}}
    return code_df, cvr_dfs, ctr_dfs


In [0]:
root_dir = './data/'
code, cvr_logs, ctr_logs = dataload(root_dir)

## Data Sample 확인

### code

In [0]:
code.shape

(1596026, 2)

In [0]:
code.head()

,CAR_SEQ,CAR_CODE
0,10000101,1626.0
1,10000123,1336.0
2,10000125,1523.0
3,10000126,2096.0
4,10000127,2096.0


### ctr_logs

In [0]:
ctr_logs.keys()

dict_keys(['car_master_read_df'])

In [0]:
ctr_logs["car_master_read_df"]['data'].shape

(1586360, 3)

In [0]:
ctr_logs["car_master_read_df"]['data'].head()

,CAR_SEQ,MEMBER_NO,REGI_DATE
0,10271232,131011,2017-02-23 오후 10:00:05
1,10201324,196303,2017-02-23 오후 10:00:20
2,10272206,192525,2017-02-23 오후 10:00:22
3,10272206,192525,2017-02-23 오후 10:00:27
4,10272593,192525,2017-02-23 오후 10:00:52


### cvr_logs

In [0]:
cvr_logs.keys()

dict_keys(['tb_call_request', 'tb_car_reservation', 'tb_car_wish', 'tb_sms'])

**tb_call_request**

In [0]:
cvr_logs["tb_call_request"]['data'].shape

(187, 3)

In [0]:
cvr_logs["tb_call_request"]['data'].head()

,MEMBER_NO,CAR_SEQ,REGI_DATE
0,100811,10001588,2016-07-11 오전 11:11:22
1,129669,10039290,2016-07-11 오전 11:11:22
2,121736,10037641,2016-07-11 오전 11:11:22
3,121736,10037641,2016-07-11 오전 11:11:22
4,129796,10058229,2016-07-11 오전 11:11:22


**tb_car_reservation**

In [0]:
cvr_logs["tb_car_reservation"]['data'].shape

(14579, 6)

In [0]:
cvr_logs["tb_car_reservation"]['data'].head()

,CAR_SEQ,RESERVATION_SEQ,MEMBER_NO,RESERVATION_DAY,RESERVATION_STATE,REGI_DATE
0,11129677,1,365489,20190910,56140,2019-09-09 오후 10:33:16
1,11417832,1,365584,20190910,56110,2019-09-09 오후 9:18:02
2,11448934,1,365716,20190909,56130,2019-09-09 오후 8:35:48
3,11467743,3,365716,20190910,56140,2019-09-09 오후 8:23:21
4,11222575,1,365584,20190910,56140,2019-09-09 오후 7:11:13


**tb_car_wish**

In [0]:
cvr_logs["tb_car_wish"]['data'].shape

(530058, 6)

In [0]:
cvr_logs["tb_car_wish"]['data'].head()

,MEMBER_NO,CAR_SEQ,MODIFY_USER,MODIFY_DATE,REGI_USER,REGI_DATE
0,365258,11532141,365258,2019-09-09 오후 11:59:48,365258,2019-09-09 오후 11:59:48
1,244877,11411913,244877,2019-09-09 오후 11:59:34,244877,2019-09-09 오후 11:59:34
2,365644,11388293,365644,2019-09-09 오후 11:57:01,365644,2019-09-09 오후 11:57:01
3,365258,11531734,365258,2019-09-09 오후 11:56:34,365258,2019-09-09 오후 11:56:34
4,244877,11295750,244877,2019-09-09 오후 11:55:23,244877,2019-09-09 오후 11:55:23


**tb_sms**

In [0]:
cvr_logs["tb_sms"]['data'].shape

(81871, 5)

In [0]:
cvr_logs["tb_sms"]['data'].head()

,SMS_SEQ,SMS_KIND,CAR_SEQ,MEMBER_NO,REGI_DATE
0,2132498,88170,11548772.0,0.0,2019-09-09 오후 11:48:44
1,2132482,88170,11589818.0,364010.0,2019-09-09 오후 11:19:22
2,2132479,88170,11595581.0,365035.0,2019-09-09 오후 11:11:39
3,2132475,88170,11555173.0,365035.0,2019-09-09 오후 11:07:05
4,2132449,88170,11593664.0,0.0,2019-09-09 오후 10:26:17


## 문제 정의

유저에게 "차종"을 추천<br>
그러나, 위의 데이터는 "CAR_SEQ": 차량 매물로 구성된 데이터셋

In [0]:
# ""~에게 ~을" 추천 문제에 대한 컬럼 명시
code_column = 'CAR_CODE'
log_column = 'MEMBER_NO'

## 전처리 전체 프로세스(Make UI Matrix)

전처리를 통해 log 데이터를 목적에 맞는 User-Item 매트릭스로 변환할 것

![preprocess](https://i.imgur.com/ZZEzVCl.png)

In [0]:
def preprocess(dfs, code_df, log_weights, threshold, log_column, code_column):

    """
    Description:
        left_df, right_df을 특정 column(forein key)을 기준으로 left merge 을 수행한다.
        수행후 생겨난 NaN 값을 제거하고 Guest 값을 제거 합니다.
    Args:
        dfs: DataFrame, 아래와 같은 구조를 가집니다.
            {
                '이름1': {'data':   DataFrame, weignt: int},
                '이름2': {'data':   DataFrame, weight: int},
                                 ...
                '이름n': {'data':   DataFrame, weight: int},
            }
            Dataframe의 columns 은 아래와 같음.[COUNT]
        code_df: DataFrame, code 정보가 들어있는 matrix
        log_weights: dict, 각 log 파일별 중요도
        threshold: int, 최소 중복 갯수.
        log_column: str, log_dfs 에서 추출해 최종 dfs에 존재할 log_columns 이름
        code_column: str, code_df 에서 추출해 최종 dfs에 존재할 code_columns 이름
    Returns:
        ui_mat, DataFrame,
        index_codes, ui_mat(Dataframe)의 indices 정보.
    """

    # dfs['COUNT'] 에 weight 값을 추가함
    dfs = add_weights(dfs, **log_weights)

    # code 정보와 log정보를 취합.
    dfs = merge_code_with_log(code_df, dfs)

    # dfs 내에 들어있는 데이터들을 하나의 데이터로 통합
    dfs = pd.concat([dfs[name]['data'] for name in dfs])

    # GUEST 을 제거. GUEST은 0번이라고 지정되어 있음.
    dfs = dfs[dfs.MEMBER_NO != 0]

    # Drop NaN Values
    dfs.dropna(inplace=True)

    # log가 threshold 이상 중복 되어야 함.
    dfs = filtering(dfs, threshold, log_column, code_column)

    # ui-matrix 생성
    ui_mat, index_codes = generate_ui_matrix(dfs, log_column, code_column)
    return ui_mat, index_codes

![img_2](https://i.imgur.com/RHFcPvK.png)

### 데이터별 가중치 부여

In [0]:
def add_weights(dfs, **names_weights):
    """
    Description:
        입력 dict인 dfs에 알맞는 weight 을 추가함.
    TODO: 그림 추가하기
    Args:
        dfs: dict, 아래 형태로 들어 있는 dictionary
        {
            '이름1': {'data':   DataFrame},
            '이름3': {'data':   DataFrame},
                        ...
            '이름n': {'data':   DataFrame},
        }
    Returns:
        dict, 아래 형태를 가진 dictionary
        {
            '이름1': {'data':   DataFrame, weignt: int},
            '이름2': {'data':   DataFrame, weight: int},
                     ...
            '이름n': {'data':   DataFrame, weight: int},
        }
    """

    assert names_weights.keys() == dfs.keys(), '입력 weights 는 log dataframe 이름과 동일 해야 합니다.'
    for name, value in names_weights.items():
        weight = names_weights[name]
        dfs[name]['weight'] = weight
    return dfs

In [0]:
# 각 데이터별로 가중치를 얼마나 줄 것인가 "Hyper Parameter" 설정
ctr_weights = {'car_master_read_df': 1.}
cvr_weights = {'tb_call_request': 1.3, 'tb_car_reservation': 1.1, 'tb_car_wish': 1.2, 'tb_sms': 1.3}

In [0]:
ctr_dfs = add_weights(ctr_logs, **ctr_weights)
cvr_dfs = add_weights(cvr_logs, **cvr_weights)

In [0]:
ctr_dfs['car_master_read_df'].keys()

dict_keys(['data', 'weight'])

In [0]:
cvr_dfs['tb_sms'].keys()

dict_keys(['data', 'weight'])

### 목적에 맞는 Code로 치환

In [0]:
def merge_code_with_log(code_df, log_dfs, on='CAR_SEQ', log_columns='MEMBER_NO'):
    """
    Description:
        코드 정보와 log 정보를 합친후 적절한 추처리를 합니다.
         필요한 정보는 추가하고 필요없는 정보를 제거합니다.
    Args:
        code_df: DataFrame, code 정보가 들어있는 dataframe
        log_dfs: dict , log 정보를 가지고 있는 dict, 아래와 같은 형태를 가집니다.
            {
                '이름1': {'data':   DataFrame, weignt: int},
                '이름2': {'data':   DataFrame, weight: int},
                                 ...
                '이름n': {'data':   DataFrame, weight: int},
            }
        on: code_df 와 log_dfs 을 병합할 column.
        log_columns: 로그 columns 중 유지해야 할 columns 을 남겨둡니다.
    Returns:
        dict, log_dfs 와 같은 형태의 dictionay가 반환.
        log_dfs 에서 [data] 부분이 변경됨.
    """

    for name, sub_dict in log_dfs.items():
        # 필요한 columns 만 가져옵니다.
        # TODO: 추후 log_columns 이 복수개가 될때 처리할수 있도록 코드 변경하기
        # TODO: REGI DATE 추가하기
        df = sub_dict['data'].loc[:, [log_columns, on, "REGI_DATE"]]

        # left merge 합니다
        merged_df = pd.merge(df, code_df, on=on, how='left')

        # CAR_SEQ column 을 제거합니다. 제거하지 않은면 연산량에 영향을 미칩니다
        df = merged_df.drop(on, axis=1)

        # 모든 행의 COUNT 값이 1을 보증합니다.
        # weight COUNT열을 weight 로 바꾸는건 COUNT에 weight를 곱해주는것과 동치입니다
        df['COUNT'] = sub_dict['weight']

        # 변경된 df을 dict 에 추가합니다.
        log_dfs[name]['data'] = df
    return log_dfs

In [0]:
ctr_dfs = merge_code_with_log(code, ctr_dfs, on='CAR_SEQ', log_columns='MEMBER_NO')
cvr_dfs = merge_code_with_log(code, cvr_dfs, on='CAR_SEQ', log_columns='MEMBER_NO')

In [0]:
ctr_dfs['car_master_read_df']["data"].head()

,MEMBER_NO,REGI_DATE,CAR_CODE,COUNT
0,131011,2017-02-23 오후 10:00:05,1260.0,1.0
1,196303,2017-02-23 오후 10:00:20,1462.0,1.0
2,192525,2017-02-23 오후 10:00:22,1175.0,1.0
3,192525,2017-02-23 오후 10:00:27,1175.0,1.0
4,192525,2017-02-23 오후 10:00:52,2683.0,1.0


In [0]:
cvr_dfs['tb_car_reservation']["data"].head()

,MEMBER_NO,REGI_DATE,CAR_CODE,COUNT
0,365489,2019-09-09 오후 10:33:16,1175.0,1.1
1,365584,2019-09-09 오후 9:18:02,1262.0,1.1
2,365716,2019-09-09 오후 8:35:48,1394.0,1.1
3,365716,2019-09-09 오후 8:23:21,1394.0,1.1
4,365584,2019-09-09 오후 7:11:13,2683.0,1.1


### 하나의 데이터로 통합

In [0]:
ctr_dfs = pd.concat([ctr_dfs[name]['data'] for name in ctr_dfs])
cvr_dfs = pd.concat([cvr_dfs[name]['data'] for name in cvr_dfs])

In [0]:
ctr_dfs.head()

,MEMBER_NO,REGI_DATE,CAR_CODE,COUNT
0,131011,2017-02-23 오후 10:00:05,1260.0,1.0
1,196303,2017-02-23 오후 10:00:20,1462.0,1.0
2,192525,2017-02-23 오후 10:00:22,1175.0,1.0
3,192525,2017-02-23 오후 10:00:27,1175.0,1.0
4,192525,2017-02-23 오후 10:00:52,2683.0,1.0


In [0]:
cvr_dfs.head()

,MEMBER_NO,REGI_DATE,CAR_CODE,COUNT
0,100811.0,2016-07-11 오전 11:11:22,1161.0,1.3
1,129669.0,2016-07-11 오전 11:11:22,1462.0,1.3
2,121736.0,2016-07-11 오전 11:11:22,1536.0,1.3
3,121736.0,2016-07-11 오전 11:11:22,1536.0,1.3
4,129796.0,2016-07-11 오전 11:11:22,1304.0,1.3


### Guest 제거

게스트는 MEMBER_NO가 0입니다. 데이터에 존재하는 Guest의 수를 확인 해 보겠습니다.

In [0]:
ctr_dfs.loc[ctr_dfs.MEMBER_NO==0,:].shape

(0, 4)

In [0]:
cvr_dfs.loc[cvr_dfs.MEMBER_NO==0,:].shape

(16986, 4)

**제거**

In [0]:
ctr_dfs = ctr_dfs.loc[ctr_dfs.MEMBER_NO != 0, :]
cvr_dfs = cvr_dfs.loc[cvr_dfs.MEMBER_NO != 0, :]

In [0]:
ctr_dfs.loc[ctr_dfs.MEMBER_NO==0,:].shape

(0, 4)

In [0]:
cvr_dfs.loc[cvr_dfs.MEMBER_NO==0,:].shape

(0, 4)

### 결측치 제거

In [0]:
print("ctr_dfs에 결측치가 존재 하는가? : \n {}".format(ctr_dfs.isna().any()))
print("cvr_dfs에 결측치가 존재 하는가? : \n {}".format(cvr_dfs.isna().any()))

ctr_dfs에 결측치가 존재 하는가? : 
 MEMBER_NO    False
REGI_DATE    False
CAR_CODE     False
COUNT        False
dtype: bool
cvr_dfs에 결측치가 존재 하는가? : 
 MEMBER_NO     True
REGI_DATE    False
CAR_CODE      True
COUNT        False
dtype: bool


In [0]:
ctr_dfs.dropna(inplace=True)
cvr_dfs.dropna(inplace=True)

In [0]:
print("ctr_dfs에 결측치가 존재 하는가? : \n {}".format(ctr_dfs.isna().any()))
print("cvr_dfs에 결측치가 존재 하는가? : \n {}".format(cvr_dfs.isna().any()))

ctr_dfs에 결측치가 존재 하는가? : 
 MEMBER_NO    False
REGI_DATE    False
CAR_CODE     False
COUNT        False
dtype: bool
cvr_dfs에 결측치가 존재 하는가? : 
 MEMBER_NO    False
REGI_DATE    False
CAR_CODE     False
COUNT        False
dtype: bool


### 일정 로그 수 이하의 데이터 제거 (k-core)

In [0]:
def filtering(matrix_df, threshold=3, *column_names):
    """
    Description:
    아래와 같은 순서로 운영됩니다
    nan 값 제거 후
    matrix_df 에서 지정된 column 의 element 중 중복된 갯수가
    특정 기준(threshold)이하인 모든 열을 제거합니다.
    최종적으로 반환되는 matrix 속 지정된 여러 column 모든 element 들은
    threshold 이상의 중복된 값을 가지고 있습니다.
    Example) 만약 threshold가 2 라면 column_names 모든 열의
    중복갯수가 2개 이상이 된 row 만 살아납니다.
    +---------+---------+
    | column1 | column2 |
    +---------+---------+
    |    a    |   가    |
    +---------+---------+        +---------+---------+
    |    b    |   나    |         | column1 | column2 |
    +---------+---------+ -----> +---------+---------+
    |    a    |   다    |         |    c    |   다    |
    +---------+---------+        +---------+---------+
    |    c    |   다    |         |    c    |   다    |
    +---------+---------+        +---------+---------+
    |    c    |   다    |
    +---------+---------+
    |    c    |   nan   |
    +---------+---------+
    Args:
        matrix_df: DataFrame, 해당 연산을 수행할 source matrix 입니다.
        threshold: int, 최종 matrix 에서 특정 column 의 동일 element 가 남아있을 최소 기준입니다.
        *column_names : str, matrix_df 에 존재하는 column 이름입니다.
            example) 'MEMBER_NO', 'MODEL_CODE'
    Returns:
        DataFrame, 해당 전처리가 적용된 matrix
    Usage:
        filterd_read_df = process_for_density(read_df, 3, 'MEMBER_NO', 'MODEL_CODE')
    """

    while True:
        mask_array = []
        for col_name in column_names:
            # 지정된 column 에서 각 element 의 중복 갯수를 Series 로 반환
            # index 는 element, value 는 count
            col_counts = matrix_df.loc[:, col_name].value_counts()

            # 중복 갯수가 기준값 이상인 element 을 가져옴.
            valid_column_values = col_counts.loc[col_counts >= threshold].index.values

            # matrix 에서 해당 element 의 위치를 나타내는 mask 생성 및 list 에 추가
            valid_bool_indice = matrix_df.loc[:, col_name].isin(valid_column_values)
            mask_array.append(valid_bool_indice.values)

        # 여러 mask 을 and 연산함, 하나의 boolean mask 을 생성.
        valid_indices = np.asarray(mask_array).all(axis=0)

        # 위 생성한 boolean mask에서 True 위치에 있는 element 만 뽑아냅니다.
        # 추출한 element 의 길이가 변하면 위 loop을 다시 수행해
        # element 갯수가 변하지 않을때 까지 수행합니다.
        if len(matrix_df) == len(matrix_df.loc[valid_indices, :]):
            break
        else:
            matrix_df = matrix_df.loc[valid_indices, :]
    return matrix_df

In [0]:
# log의 수가 3회 이하로 발생한 MEMBER_NO, CAR_CODE 순환적 제거
ctr_threshold = 3
cvr_threshold = 3

In [0]:
ctr_dfs = filtering(ctr_dfs, ctr_threshold, log_column, code_column)
cvr_dfs = filtering(cvr_dfs, cvr_threshold, log_column, code_column)

### User-Item 매트릭스로 변환

In [0]:
def generate_ui_matrix(matrix_df, index_name, columns_name):
    """
    Description :
        matrix_df 에 특정열을 이용해 value 가 중복된 갯수인
        pivot table 을 생성합니다.
        +--------+---------+---------+
        |        | column0 | column1 |
        +--------+---------+---------+
        | index0 |   32    |    4    | <- index0, column1 의 중복 갯수 : 4
        +--------+---------+---------+
        | index1 |    3    |    2    |
        +--------+---------+---------+
        | index2 |    7    |    1    |
        +--------+---------+---------+
    Args:
        matrix_df: DataFrame,
        index_name: str, matrix_df 에 존재하는 column 이름,
            해당 column 은 pivot table 에서 `행(row)`에 해당한다.
        columns_name: str, matrix_df 에 존재하는 column 이름,
    Returns: DataFrame,
    """
    # 최종 열이 중복된 갯수가 될수 있도록 count 합니다.
    read_count = matrix_df.groupby(by=[index_name, columns_name]).sum().reset_index()
    read_count.columns = [index_name, columns_name, "COUNT"]

    #값을 조정합니다.
    # read_count["COUNT"] = np.log(read_count["COUNT"])

    # pivot table 을 생성합니다.
    ui_matrix = read_count.pivot_table(values="COUNT", index=columns_name,
                                       columns=index_name, aggfunc='sum',
                                       fill_value=0)
    ui_matrix.index = ui_matrix.index.astype('int')
    ui_matrix.columns = ui_matrix.columns.astype('int')

    # pivot table index 순서
    codes = ui_matrix.index.astype(np.int).values

    return ui_matrix, codes

In [0]:
ctr_ui_mat, ctr_index_codes = generate_ui_matrix(ctr_dfs, log_column, code_column)
cvr_ui_mat, cvr_index_codes = generate_ui_matrix(cvr_dfs, log_column, code_column)

## CVR 데이터와 CTR 데이터 합치기

In [0]:
# Merge CVR + CTR
ui_mat = ctr_ui_mat.add(cvr_ui_mat, fill_value=0).fillna(0)
index_code = ui_mat.index

## 유사도 계산

위에서 만든 User-Item 매트릭스를 통해 어떻게 추천 항목을 추출할 수 있을까요.<br>
우리는 아래와 같은 프로세스로 추천 항목을 추천하고자 합니다. <br>
(Item-Based Collaborative Filtering)

!["추천프로세스"](https://i.imgur.com/wOYOX68.png)

여기서, "유사도 계산"는 어떻게 할 수 있을까요.<br>
두 차종을 좋아하는 유저 집합의 선호가 비슷하다면, 두 차종은 "유사한 취향을 가진 유저들이 좋아하는 차종"이라고 볼 수 있습니다.<br>
계산 방법은 다음 슬라이드를 참조해주세요.


[유사도 계산 방법에 대하여](./similarity.ipynb)

이러한 유사도 계산 방식을 해당 데이터에 맞게 적용하면<br>

여기서는 유사도 계산식을 기본값으로 Jaccard로 설정하였습니다.

In [0]:
def calculate_similarity_matrix(ui_matrix, similarity='jaccard'):

    """
    Description :
        jarcarrd Similarity 을 계산합니다.
        계산방식은 아래와 같습니다.
        Similarity = interscetion / union
    Args:
        ui_matrix: Ndarray
        similarity: str, 어떤 similarity 을 사용해 추출할 것인지 결정합니다.
    Returns:
        Ndarray, symetric matrix
    """
    # broadcast을 이용하기 위해 ui_matrix의 shape 을 변형합니다.
    row_data = np.expand_dims(ui_matrix, axis=0)
    column_data = np.expand_dims(ui_matrix, axis=1)

    if similarity == 'jaccard':
        # or 연산을 수행한 후, True인 값의 갯수를 파악합니다.
        union = np.sum(np.logical_or(row_data, column_data), axis=-1)

        # and 연산을 수행한 후, True인 값의 갯수를 파악합니다.
        intersection = np.sum(np.logical_and(row_data, column_data), axis=-1)
        return intersection / union

    elif similarity == 'pearson':
        # pearson similarity 을 계산합니다. 
        sim_mat = np.corrcoef(ui_matrix)
        sim_mat = np.nan_to_num(sim_mat)
        return sim_mat

    else:
        print('아직 구현 되지 않은 유사도 방식입니다. 구현된 유사도는 아래와 같습니다.')
        print('[ jaccard ] [ pearson ]')
        raise NotImplementedError
    # similarity 을 계산해 반환합니다.

In [0]:
# Generate pearson similarity
sim_mat=calculate_similarity_matrix(ui_mat)

## 계산된 Sim_mat을 통한 추천 도출 및 모델 저장

In [0]:
def recomsys_savedmodel(similarity_matrix, model_table, save_dir, num_recommand=20):
    """
    Description:
        similarity matrix 에서 input 과 가장 유사성이 높은 index을 반환합니다.
        model table 을 이용해 추천 받은 index 을 item number 로 변환합니다.
        해당 함수는 tensorflow graph , variable 을
        tensorflow serving 을 수행할수 있도록 .pb 파일과 variable 을 특정 폴더 위치에 저장합니다.
        save_dir 이 만약 /tmp/chachacha 라면 .pb 파일 , variable 은
        /tmp/chachacha/1 에 저장됩니다. 뒤에 숫자는 버전을 의미합니다.
        만약 /tmp/chachacha/1 이 있는 상태에서 다시 해당 함수를 수행하면
        자동으로 /tmp/chachacha/2 폴더를 생성하고 해당 폴더에 .pb, variable 을 저장합니다.
    Args:
        similarity_matrix : Ndarray, similarity 가 계산된 matrix입니다.
        model_table : Ndarray, similarity_matrix index 순서와 matching 되는
            item number 가 저장된 ndarray
        save_dir : str, serving 에 필요한 파일이 저장되는 공간
            example) /tmp/chachacha
        num_recommand : int, 추천받을 아이탬의 갯수
    Return: None
    """
    next_version = find_latest_version(save_dir) + 1
    export_dir = os.path.join(save_dir, str(next_version))
    builder = tf.saved_model.builder.SavedModelBuilder(export_dir=export_dir)

    with tf.Session() as sess:
        # model code 을 입력받습니다.
        target_codes = tf.placeholder(
            dtype=tf.int64, shape=[None], name='model_codes')

        # Similarity 계산을 하는 Matrix을 변수 텐서로 받습니다
        tf_similarity_matrix = tf.Variable(
            tf.constant(similarity_matrix), name='similarity_matrix')

        # model_table은 index 와 model_codes 가 기록되어 있습니다
        tf_model_codes = tf.Variable(tf.constant(model_table), name='model_codes')

        # 최소 1개 이상의 code 을 index 값으로 변화 시킵니다.
        model_indices = tf.map_fn(lambda x: tf.where(tf.equal(tf_model_codes, x))[0], target_codes)

        # 필요 없는 list 을 제거합니다 [[0,1]] -> [0,1]
        model_indices = model_indices[:, 0]

        # 해당 아이탬의 유사도를 가져옵니다.
        recom_values_list = tf.gather(tf_similarity_matrix, model_indices)
        recom_values_list = tf.cast(recom_values_list, tf.float32)
        recom_values = tf.reduce_sum(recom_values_list, axis=0)

        # 유사도가 높은 k 개의 아이탬의 value 값과 index 값을 추출합니다.
        top_k_values, top_k_indices = tf.nn.top_k(tf.cast(recom_values, tf.float32), num_recommand)

        # index 을 model_code 로 변환합니다.
        top_k_codes = tf.gather(tf_model_codes, top_k_indices)

        # 초기값을 가져옵니다.
        sess.run(tf.global_variables_initializer())
        predict_signature_def = (
            tf.saved_model.signature_def_utils.predict_signature_def(
                {"x": target_codes},
                {"y": top_k_codes}))

        signature_def_map = {
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                predict_signature_def
        }

        builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map=signature_def_map)
        builder.save()

### 마지막 버젼의 모델만 가져오기

In [0]:
def find_latest_version(save_dir):
    """
    Description :
        특정 폴더이름에서 값이 가장 큰 폴더의 이름을 반환합니다.
    Args:
        save_dir: str, example) /tmp/chachacha
    Returns: int, 폴더 명중에 가장 값이 큰 폴더 ,만약 폴더가 없으면 0을 반환
    """

    # valid check : 해당 코드가 숫자인지 확인한다.
    versions = [int(version) for version in os.listdir(save_dir) if version.isdigit()]
    if len(versions) > 0:
        return max(versions)
    else:
        os.makedirs(save_dir, exist_ok=True)
        return 0

In [0]:
def send_req(codes, url='http://58.236.168.36:8686/v1/models/chachacha:predict'):
    """
    Description:
        해당코드는 url 에 POST 방식으로 request 을 날리는 코드 입니다.
    Args:
        codes: list, 모델 number 가 들어있는 리스트
        url: str,  tensorlflow serving 은 url 방식이 고정되어 있습니다.
    Return:
        ret_codes, list , 추천 아이탬이 들어있는 list
    """

    data = '{"instances": %s}' % str(codes)
    response = requests.post(url, data=data)
    ret_codes = response.json()['predictions']
    return ret_codes

In [0]:
recomsys_savedmodel(sim_mat, index_code, '/tmp/chachacha',  num_recommand=20)